# 1 Initialize

In [1]:
visible_devices = 1
print("Visible devices:", visible_devices)
import datetime
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing os...")
import os
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing re...")
import re
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timer from timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting env variables for tf import (only device", visible_devices, "will be available)...")
os.environ["CUDA_VISIBLE_DEVICES"] = str(visible_devices)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing numpy...")
import numpy as np
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing matplotlib...")
from matplotlib import pyplot as plt
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing sys...")
import sys
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing h5py...")
import h5py
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing importlib.util...")
import importlib.util
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing json...")
import json
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing pandas...")
import pandas as pd
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing random...")
import random
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing scipy utils...")
from scipy.stats import norm, chi2, kstwo, kstwobign, ks_2samp, moment
from scipy.special import kolmogorov
from scipy.optimize import minimize, curve_fit, root, bisect
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing subprocess...")
import subprocess
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow...")
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow_probability...")
import tensorflow_probability as tfp
tfd = tfp.distributions
print("Tensorflow probability version:", tfp.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tqdm...")
from tqdm import tqdm
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing typing...")
from typing import List, Tuple, Dict, Callable, Union, Optional, Any, Type
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting tf configs...")
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing GMetrics module...")

sys.path.insert(0,'../utils_func/')
import MixtureDistributions # type: ignore

sys.path.insert(0,'../../')
import GenerativeModelsMetrics as GMetrics # type: ignore
from GenerativeModelsMetrics.utils import se_mean, se_std

paper_fig_dir = "../../../NormalizingFlows/papers/NFHD/figures/"

def get_gpu_info():
    try:
        gpu_info = subprocess.check_output(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv,noheader"]).decode('utf-8')
        return gpu_info.strip().split('\n')
    except Exception as e:
        print(e)
        return None
gpu_models = get_gpu_info()
if gpu_models:
    training_device = gpu_models[eval(os.environ["CUDA_VISIBLE_DEVICES"])]
    print("Successfully loaded GPU model: {}".format(training_device))
else:
    training_device = 'undetermined'
    print("Failed to load GPU model. Defaulting to 'undetermined'.")
    
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "All done.")

Visible devices: 1
2023-08-25 13:27:21.452397: Importing os...
2023-08-25 13:27:21.452571: Importing re...
2023-08-25 13:27:21.452706: Importing timer from timeit...
2023-08-25 13:27:21.452876: Setting env variables for tf import (only device 1 will be available)...
2023-08-25 13:27:21.453134: Importing numpy...
2023-08-25 13:27:21.607258: Importing matplotlib...
2023-08-25 13:27:21.946350: Importing sys...
2023-08-25 13:27:21.946555: Importing h5py...
2023-08-25 13:27:21.966550: Importing importlib.util...
2023-08-25 13:27:21.966652: Importing json...
2023-08-25 13:27:21.966719: Importing pandas...
2023-08-25 13:27:22.160378: Importing random...
2023-08-25 13:27:22.160517: Importing scipy utils...
2023-08-25 13:27:22.434849: Importing subprocess...
2023-08-25 13:27:22.434978: Importing tensorflow...
Tensorflow version: 2.12.0
2023-08-25 13:27:24.292073: Importing tensorflow_probability...
Tensorflow probability version: 0.20.1
2023-08-25 13:27:24.550239: Importing timeit...
2023-08-25

# 2.1 Define distribution and inputs

In [2]:
ndims = 10
seed = 0
epsilon = 0.005
loc1 = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
scale1 = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
loc2 = np.array(np.random.uniform(-epsilon,epsilon,ndims),dtype=np.float64)
scale2 = np.array(np.random.uniform(1-epsilon,1+epsilon,ndims),dtype=np.float64)
dist_1 = tfd.Independent(tfp.distributions.Normal(loc=loc1, scale=scale1), reinterpreted_batch_ndims=1)
dist_2 = tfd.Independent(tfp.distributions.Normal(loc=loc2, scale=scale2), reinterpreted_batch_ndims=1)
print(dist_1.dtype, dist_2.dtype)
print('Types of distributions: ', type(dist_1), type(dist_2))
print('Dtypes of distributions: ', dist_1.dtype, dist_2.dtype)
print('Batch shapes of distributions: ', dist_1.batch_shape, dist_2.batch_shape)
print('Event shapes of distributions: ', dist_1.event_shape, dist_2.event_shape)

<dtype: 'float64'> <dtype: 'float64'>
Types of distributions:  <class 'tensorflow_probability.python.distributions.independent.Independent'> <class 'tensorflow_probability.python.distributions.independent.Independent'>
Dtypes of distributions:  <dtype: 'float64'> <dtype: 'float64'>
Batch shapes of distributions:  () ()
Event shapes of distributions:  (10,) (10,)


In [3]:
#ncomp = 3
#ndims = 10
#seed = 0
#epsilon = 0.005
#loc1 = np.array(np.zeros([ncomp,ndims],dtype=np.float64),dtype=np.float64)
#scale1 = np.array(np.ones([ncomp,ndims],dtype=np.float64),dtype=np.float64)
#loc2 = np.array(np.random.uniform(-epsilon,epsilon,[ncomp,ndims]),dtype=np.float64)
#scale2 = np.array(np.random.uniform(-epsilon,epsilon,[ncomp,ndims]),dtype=np.float64)
#comp1 = []
#for i in range(ncomp):
#    comp1.append(tfd.MultivariateNormalDiag(loc=loc1[i],scale_diag=scale1[i]))
#comp2 = []
#for i in range(ncomp):
#    comp2.append(tfd.MultivariateNormalDiag(loc=loc2[i],scale_diag=scale2[i]))
#probs = np.random.sample(ncomp)
#dist_1 = tfd.Mixture(
#            cat=tfd.Categorical(probs = probs),
#            components = comp1,
#            validate_args = True)
#dist_2 = tfd.Mixture(
#            cat=tfd.Categorical(probs = probs),
#            components = comp2,
#            validate_args = True)
#print('Types of distributions: ', type(dist_1), type(dist_2))
#print('Dtypes of distributions: ', dist_1.dtype, dist_2.dtype)
#print('Batch shapes of distributions: ', dist_1.batch_shape, dist_2.batch_shape)
#print('Event shapes of distributions: ', dist_1.event_shape, dist_2.event_shape)

In [4]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 10,
                                                      batch_size = 10000,
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)
print("small_sample",TwoSampleTestInputs_tf.small_sample)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 100000
batch_size 10000
niter 10
niter * batch_size 100000
small_sample True


In [5]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            progress_bar = True,
                            verbose = True)
SWDMetric_tf = GMetrics.SWDMetric(TwoSampleTestInputs_tf,
                                  progress_bar = True,
                                  verbose = True)
FNMetric_tf = GMetrics.FNMetric(TwoSampleTestInputs_tf,
                                progress_bar = True,
                                verbose = True)
LRMetric_tf = GMetrics.LRMetric(TwoSampleTestInputs_tf,
                                progress_bar = True,
                                verbose = True)

progress_bar is disabled when using tensorflow mode.
progress_bar is disabled when using tensorflow mode.
progress_bar is disabled when using tensorflow mode.
progress_bar is disabled when using tensorflow mode.


# Compute tests Numpy

## KS Test

In [6]:
KSTest_tf.Test_np()
KSTest_tf.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
niter = 10
batch_size = 10000
Running numpy KS tests...


Two-sample test calculation completed in 0.40222117211669683 seconds.


{'statistic_lists': array([[0.0112, 0.014 , 0.0112, 0.0114, 0.0178, 0.0134, 0.0185, 0.0101,
         0.0093, 0.0086],
        [0.0066, 0.019 , 0.0129, 0.0183, 0.0215, 0.0142, 0.0084, 0.0111,
         0.0125, 0.0134],
        [0.0221, 0.0184, 0.0205, 0.0117, 0.012 , 0.0135, 0.0128, 0.0108,
         0.0082, 0.0098],
        [0.0129, 0.0085, 0.0186, 0.018 , 0.0163, 0.0091, 0.01  , 0.0093,
         0.0083, 0.0203],
        [0.0071, 0.0182, 0.0104, 0.0123, 0.0095, 0.0081, 0.0143, 0.0127,
         0.0109, 0.008 ],
        [0.0215, 0.0147, 0.0115, 0.0265, 0.0097, 0.0091, 0.0072, 0.012 ,
         0.0161, 0.0096],
        [0.0118, 0.0138, 0.008 , 0.0124, 0.0075, 0.0153, 0.0154, 0.0129,
         0.0071, 0.0097],
        [0.0109, 0.021 , 0.011 , 0.0152, 0.0105, 0.0129, 0.0064, 0.0088,
         0.0131, 0.0159],
        [0.0104, 0.0122, 0.0085, 0.0121, 0.0111, 0.0065, 0.0121, 0.0128,
         0.0138, 0.017 ],
        [0.0239, 0.0144, 0.0094, 0.0182, 0.0105, 0.0099, 0.0128, 0.011 ,
         0.0117, 

## SWD Test

In [7]:
SWDMetric_tf.Test_np()
SWDMetric_tf.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
niter = 10
batch_size = 10000
Running numpy SWD calculation...
Two-sample test calculation completed in 4.7728085508570075 seconds.


{'metric_lists': array([[0.02204142, 0.01586819, 0.02505003, 0.02319684, 0.02037962,
         0.02386106, 0.01537622, 0.01209821, 0.02915753, 0.0161557 ,
         0.02183029, 0.01547827, 0.02035983, 0.01375458, 0.01703099,
         0.02884583, 0.0155018 , 0.01877795, 0.01209744, 0.01908813,
         0.02195479, 0.01888886, 0.01809339, 0.0197328 , 0.02714779,
         0.01205071, 0.01738729, 0.0184274 , 0.0277889 , 0.01771307,
         0.02026116, 0.00867615, 0.01699459, 0.02501348, 0.01377203,
         0.01521871, 0.0168115 , 0.02138491, 0.01449245, 0.01602701,
         0.01636821, 0.02874294, 0.01554554, 0.01660892, 0.0115483 ,
         0.01993823, 0.01470065, 0.0208475 , 0.01782927, 0.02156867,
         0.01831997, 0.02152972, 0.01378837, 0.01405687, 0.0160503 ,
         0.03357668, 0.01844979, 0.02471944, 0.03348806, 0.01475405,
         0.02014797, 0.02045557, 0.02141863, 0.01948375, 0.0130159 ,
         0.02511411, 0.03066222, 0.01387797, 0.01858688, 0.02260815,
         0.0157154

## FN Test

In [8]:
FNMetric_tf.Test_np()
FNMetric_tf.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
niter = 10
batch_size = 10000
Running numpy FN calculation...
Two-sample test calculation completed in 0.013064120896160603 seconds.


{'metric_list': array([0.13075619, 0.13247258, 0.15449843, 0.129129  , 0.15546639,
        0.15634946, 0.128004  , 0.13520122, 0.12383472, 0.15304271])}

## LR Test

In [9]:
LRMetric_tf.Test_np()
LRMetric_tf.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
niter = 10
batch_size = 10000
Running numpy LR calculation...
Two-sample test calculation completed in 0.05025598406791687 seconds.


{'logprob_ref_ref_sum_list': array([-141682.0397089 , -141777.65703793, -141992.18458847,
        -141748.35729362, -141842.03710355, -141943.85687927,
        -142084.38244331, -141481.89793912, -141478.49212031,
        -141832.06445463]),
 'logprob_ref_alt_sum_list': array([-143013.96548793, -142113.59688427, -142550.19122945,
        -142307.57380389, -142185.1623157 , -142182.96393057,
        -142315.96231988, -142051.75159238, -142003.51795397,
        -141928.24092813]),
 'logprob_alt_alt_sum_list': array([-143007.86798318, -142113.9213449 , -142546.67401549,
        -142305.82602736, -142181.22789654, -142180.2278826 ,
        -142315.05746742, -142050.00954529, -141999.66966865,
        -141927.87999389]),
 'lik_ratio_list': array([12.19500949, -0.64892128,  7.03442791,  3.49555306,  7.86883832,
         5.47209595,  1.80970492,  3.48409417,  7.69657064,  0.72186846]),
 'lik_ratio_norm_list': array([ 0.12195009, -0.00648921,  0.07034428,  0.03495553,  0.07868838,
         0.0

# Compute tests TensorFlow

## KS Test

In [10]:
KSTest_tf.Test_tf()
KSTest_tf.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 10
batch_size = 10000


KS tests calculation completed in 3.0043447287753224 seconds.


{'statistic_lists': array([[0.0112    , 0.014     , 0.0112    , 0.0114    , 0.0178    ,
         0.01340002, 0.0185    , 0.01010001, 0.00929999, 0.0086    ],
        [0.00660001, 0.01899999, 0.01289999, 0.0183    , 0.02150005,
         0.0142    , 0.00840002, 0.01110005, 0.01250005, 0.01340002],
        [0.02209997, 0.01840001, 0.0205    , 0.0117    , 0.01200002,
         0.01349998, 0.01280004, 0.0108    , 0.00819999, 0.00980002],
        [0.01290001, 0.00849998, 0.01860002, 0.01800001, 0.01630002,
         0.00910002, 0.00999999, 0.00929999, 0.00830001, 0.0203    ],
        [0.00710002, 0.01819998, 0.0104    , 0.01230001, 0.0095    ,
         0.00810003, 0.01430002, 0.01270001, 0.01090002, 0.008     ],
        [0.02150002, 0.01470006, 0.0115    , 0.02650002, 0.0097    ,
         0.00909996, 0.0072    , 0.01199996, 0.01610005, 0.00960001],
        [0.01179999, 0.01380002, 0.00799996, 0.01240003, 0.00749999,
         0.01530001, 0.01540005, 0.0129    , 0.00709999, 0.0097    ],
        

## SWD Test

In [11]:
SWDMetric_tf.Test_tf()
SWDMetric_tf.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 10
batch_size = 10000
SWD metric calculation completed in 1.8032825542613864 seconds.


{'metric_lists': array([[0.01999089, 0.02468832, 0.02263426, 0.02411087, 0.02088434,
         0.02516768, 0.00895595, 0.02683157, 0.03342044, 0.01454918,
         0.01259425, 0.02338609, 0.01870304, 0.02363907, 0.02544613,
         0.00965089, 0.01562472, 0.03035842, 0.01547713, 0.01972453,
         0.0326089 , 0.01805841, 0.02496615, 0.03265846, 0.01821534,
         0.02860542, 0.03806234, 0.02507176, 0.02366488, 0.02592573,
         0.01990642, 0.01796745, 0.02436539, 0.02400841, 0.020937  ,
         0.01863841, 0.01841204, 0.00897293, 0.01752079, 0.02362915,
         0.0218083 , 0.028873  , 0.02626006, 0.01952471, 0.01949477,
         0.01740659, 0.01805062, 0.03420691, 0.01882766, 0.0245403 ,
         0.01142402, 0.01740921, 0.0174548 , 0.01642837, 0.02280942,
         0.01571016, 0.02531879, 0.01856477, 0.01787565, 0.02265859,
         0.02611596, 0.0138157 , 0.02129888, 0.01671386, 0.01369221,
         0.02817218, 0.01676303, 0.02195189, 0.01452782, 0.01927528,
         0.0206573

## FN Test

In [12]:
FNMetric_tf.Test_tf()
FNMetric_tf.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 10
batch_size = 10000
FN metric calculation completed in 1.124456130899489 seconds.


{'metric_list': array([[0.13075619, 0.13247258, 0.15449843, 0.129129  , 0.15546639,
         0.15634946, 0.128004  , 0.13520122, 0.12383472, 0.15304271]])}

## LR Test

In [13]:
LRMetric_tf.Test_tf()
LRMetric_tf.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
Running TF FN calculation...
niter = 10
batch_size = 10000


lik_ratio = Tensor("mul:0", shape=(), dtype=float64)
LR metric calculation completed in 0.8320877905935049 seconds.


{'logprob_ref_ref_sum_list': array([-141682.0397089 , -141777.65703793, -141992.18458847,
        -141748.35729362, -141842.03710355, -141943.85687927,
        -142084.38244331, -141481.89793912, -141478.49212031,
        -141832.06445463]),
 'logprob_ref_alt_sum_list': array([-143013.96548793, -142113.59688427, -142550.19122945,
        -142307.57380389, -142185.1623157 , -142182.96393057,
        -142315.96231988, -142051.75159238, -142003.51795397,
        -141928.24092813]),
 'logprob_alt_alt_sum_list': array([-143007.86798318, -142113.9213449 , -142546.67401549,
        -142305.82602736, -142181.22789654, -142180.2278826 ,
        -142315.05746742, -142050.00954529, -141999.66966865,
        -141927.87999389]),
 'lik_ratio_list': array([12.19500949, -0.64892128,  7.03442791,  3.49555306,  7.86883832,
         5.47209595,  1.80970492,  3.48409417,  7.69657064,  0.72186846]),
 'lik_ratio_norm_list': array([ 0.12195009, -0.00648921,  0.07034428,  0.03495553,  0.07868838,
         0.0